In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import json
import cv2
from matplotlib.animation import FuncAnimation
from IPython.display import display, HTML
import numpy as np
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle

# Step 1: Connect to the database
db_path = "/Users/arijitdasgupta/Desktop/projects/red_green_site/backend/instance/pre_pilot_v0_redgreen.db"  # Update with the path to your .db file
engine = create_engine(f"sqlite:///{db_path}")  # Assuming SQLite


# Step 2: Load session data and filter for complete sessions
session_query = """
    SELECT id AS session_id, randomized_trial_order
    FROM redgreen_session
    WHERE completed = 1
"""
session_df = pd.read_sql(session_query, engine)

# Step 3: Parse randomized_trial_order and map global_trial_name
def parse_randomized_order(order_json):
    try:
        return json.loads(order_json)
    except json.JSONDecodeError:
        return []

# Add parsed randomized_trial_order to the DataFrame
session_df["parsed_order"] = session_df["randomized_trial_order"].apply(parse_randomized_order)

# Step 4: Load trial data and map `global_trial_name`
trial_query = """
    SELECT id AS trial_id, session_id, trial_index, counterbalance
    FROM trial
    WHERE trial_type != 'ftrial' AND completed = 1
"""
trial_df = pd.read_sql(trial_query, engine)

# Merge trials with session randomized order
merged_df = pd.merge(trial_df, session_df, left_on="session_id", right_on="session_id")

# Assign global_trial_name based on trial_index and parsed_order
def get_global_trial_name(row):
    try:
        return row["parsed_order"][row["trial_index"]]
    except IndexError:
        return None
# not needed for pilot
merged_df["global_trial_name"] = merged_df.apply(get_global_trial_name, axis=1)
merged_df = merged_df.drop(columns=['randomized_trial_order'])
merged_df = merged_df.drop(columns=['parsed_order'])

# Step 5: Load keystate data and filter for valid trials
valid_trial_ids = merged_df["trial_id"].dropna().tolist()

keystate_query = f"""
    SELECT ks.frame, ks.f_pressed, ks.j_pressed, ks.trial_id
    FROM keystate ks
    WHERE ks.trial_id IN ({', '.join(map(str, valid_trial_ids))})
"""
keystate_df = pd.read_sql(keystate_query, engine)

# Merge keystate data with global_trial_name
keystate_df = pd.merge(keystate_df, merged_df[["trial_id", "global_trial_name", "counterbalance"]], on="trial_id")

# Step 6: Process the data
# Convert boolean to integer for aggregation
keystate_df["f_pressed"] = keystate_df["f_pressed"].astype(int)
keystate_df["j_pressed"] = keystate_df["j_pressed"].astype(int)

keystate_df["red"] = ((keystate_df["counterbalance"] == 0) & (keystate_df["f_pressed"] == 1) & (keystate_df["j_pressed"] == 0)
                      | (keystate_df["counterbalance"] == 1) & (keystate_df["f_pressed"] == 0) & (keystate_df["j_pressed"] == 1)).astype(int)
keystate_df["green"] = ((keystate_df["counterbalance"] == 0) & (keystate_df["j_pressed"] == 1) & (keystate_df["f_pressed"] == 0)
                        | (keystate_df["counterbalance"] == 1) & (keystate_df["j_pressed"] == 0) & (keystate_df["f_pressed"] == 1)).astype(int)
keystate_df["uncertain"] = ((keystate_df["j_pressed"] == 0) & (keystate_df["f_pressed"] == 0)
                            | (keystate_df["j_pressed"] == 1) & (keystate_df["f_pressed"] == 1)).astype(int)


# Group by global_trial_name and frame, calculate mean
aggregated_df = (
    keystate_df.groupby(["global_trial_name", "frame"])
    .mean(numeric_only=True)
    .reset_index()
)

path_to_data = '/Users/arijitdasgupta/Desktop/projects/mental_physics_trials/pre_pilot'


def plot_trial_pairs(trial_names, aggregated_df, path_to_data):
    # Filter unique pairs
    pairs = [name[:-1] for name in trial_names if name[-1] == 'a']  # Get the unique "base names"
    
    for pair in pairs:
        trial_a = f"{pair}a"
        trial_b = f"{pair}b"
        
        # Create a figure for the pair
        fig = plt.figure(figsize=(12, 8))  # Adjust size as needed
        gs = GridSpec(2, 1, height_ratios=[1, 1], figure=fig)  # Two rows, one column per pair

        # Plot trial_a (top)
        ax1 = fig.add_subplot(gs[0, 0])
        plot_with_video(trial_a, aggregated_df[aggregated_df["global_trial_name"] == trial_a], ax=ax1)

        # Plot trial_b (bottom)
        ax2 = fig.add_subplot(gs[1, 0])
        plot_with_video(trial_b, aggregated_df[aggregated_df["global_trial_name"] == trial_b], ax=ax2)

        # Show the figure
        plt.suptitle(f"Pair: {trial_a} and {trial_b}", fontsize=16)
        plt.tight_layout(rect=[0, 0, 1, 0.95])  # Adjust layout to fit title
        plt.show()
        
def plot_with_video(trial_name, trial_data):
    # Path to the video
    video_path = f"{path_to_data}/{trial_name}/low_res_obs.mp4"

    print(f"Opening video: {video_path}")
    
    # Open the video and preload all frames
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Could not open video for {trial_name}")
        return None
    
    fps = cap.get(cv2.CAP_PROP_FPS)

    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        frames.append(frame_rgb)
    cap.release()

    # Check if frames were loaded
    if len(frames) == 0:
        print(f"No frames loaded for video {trial_name}")
        return None

    # Prepare the plot
    fig = plt.figure(figsize=(8, 4))  # Adjust the figure size as needed
    gs = fig.add_gridspec(1, 2, width_ratios=[1, 1])  # Create a grid spec for side-by-side layout

    # Line plot
    ax = fig.add_subplot(gs[0, 0])
    ax.set_title(f"Aggregate Red-Green Judgments {trial_name}")
    ax.set_xlabel("Frame")
    ax.set_ylabel("Proportions")
    ax.set_xlim(trial_data["frame"].min(), trial_data["frame"].max())
    ax.set_ylim(0, 1)  # Assuming proportions are between 0 and 1
    ax.grid(True)

    red_line, = ax.plot([], [], label="Red", color="red")
    green_line, = ax.plot([], [], label="Green", color="green")
    uncertain_line, = ax.plot([], [], label="Uncertain", color="blue")
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)  # Adjust bbox_to_anchor and ncol as needed
    ax.set_box_aspect(0.5)  # Aspect ratio: height = 0.5 * width (adjust value as needed)


    # Video subplot
    video_ax = fig.add_subplot(gs[0, 1])
    video_ax.axis("off")  # Turn off axis for the video frame

    # Initialize video frame placeholder
    frame_image = video_ax.imshow(frames[0])  # Start with the first frame

    # Function to update the plot and video frame
    def update(frame_idx):
        # Update the plot
        current_data = trial_data[trial_data["frame"] <= frame_idx]
        red_line.set_data(current_data["frame"], current_data["red"])
        green_line.set_data(current_data["frame"], current_data["green"])
        uncertain_line.set_data(current_data["frame"], current_data["uncertain"])

        # Update the video frame
        if frame_idx < len(frames):
            frame_image.set_array(frames[frame_idx])  # Update the video frame
        return red_line, green_line, uncertain_line, frame_image

    # Create the animation
    ani = FuncAnimation(
        fig,
        update,
        frames=len(frames),
        interval=1000 / fps,
        blit=False
    )

    # Convert animation to HTML5 video
    html_video = ani.to_html5_video()
    plt.close(fig)  # Close the plot to avoid rendering static images in Jupyter
    return HTML(html_video)


for trial_name in aggregated_df["global_trial_name"].unique()[:2]:
    trial_data = aggregated_df[aggregated_df["global_trial_name"] == trial_name]
    display(plot_with_video(trial_name, trial_data))



Opening video: /Users/arijitdasgupta/Desktop/projects/mental_physics_trials/pre_pilot/E1-1a/low_res_obs.mp4


Opening video: /Users/arijitdasgupta/Desktop/projects/mental_physics_trials/pre_pilot/E1-1b/low_res_obs.mp4


In [114]:
def plot_pair_with_video(trial_a_name, trial_a_data, trial_b_name, trial_b_data):
    # Path to the videos
    video_a_path = f"{path_to_data}/{trial_a_name}/high_res_obs.mp4"
    video_b_path = f"{path_to_data}/{trial_b_name}/high_res_obs.mp4"
    
    # Open the videos and preload all frames
    cap_a = cv2.VideoCapture(video_a_path)
    cap_b = cv2.VideoCapture(video_b_path)
    if not cap_a.isOpened() or not cap_b.isOpened():
        print(f"Error: Could not open video for {trial_a_name} or {trial_b_name}")
        return None
    
    fps_a = cap_a.get(cv2.CAP_PROP_FPS)
    fps_b = cap_b.get(cv2.CAP_PROP_FPS)

    frames_a, frames_b = [], []
    while True:
        ret_a, frame_a = cap_a.read()
        ret_b, frame_b = cap_b.read()
        if not ret_a or not ret_b:
            break
        frames_a.append(cv2.cvtColor(frame_a, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
        frames_b.append(cv2.cvtColor(frame_b, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB
    cap_a.release()
    cap_b.release()

    # Check if frames were loaded
    if len(frames_a) == 0 or len(frames_b) == 0:
        print(f"No frames loaded for videos {trial_a_name} or {trial_b_name}")
        return None

    # Prepare the plot
    fig = plt.figure(figsize=(11, 6))  # Adjust the figure size as needed
    gs = fig.add_gridspec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1])  # Create a grid spec for two rows and two columns

    # Line plot for trial_a
    ax_a = fig.add_subplot(gs[0, 0])
    ax_a.set_title(f"Red-Green Judgments: {trial_a_name}")
    ax_a.set_xlabel("Frame")
    ax_a.set_ylabel("Proportions")
    ax_a.set_xlim(trial_a_data["frame"].min(), trial_a_data["frame"].max())
    ax_a.set_ylim(0, 1)  # Assuming proportions are between 0 and 1
    ax_a.grid(True)

    red_line_a, = ax_a.plot([], [], label="Red", color="red")
    green_line_a, = ax_a.plot([], [], label="Green", color="green")
    uncertain_line_a, = ax_a.plot([], [], label="Uncertain", color="blue")
    # ax_a.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)
    ax_a.set_box_aspect(0.5)

    # Line plot for trial_b
    ax_b = fig.add_subplot(gs[0, 1])
    ax_b.set_title(f"Red-Green Judgments: {trial_b_name}")
    ax_b.set_xlabel("Frame")
    ax_b.set_ylabel("Proportions")
    ax_b.set_xlim(trial_b_data["frame"].min(), trial_b_data["frame"].max())
    ax_b.set_ylim(0, 1)  # Assuming proportions are between 0 and 1
    ax_b.grid(True)

    red_line_b, = ax_b.plot([], [], label="Red", color="red")
    green_line_b, = ax_b.plot([], [], label="Green", color="green")
    uncertain_line_b, = ax_b.plot([], [], label="Uncertain", color="blue")
    # ax_b.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)
    ax_b.set_box_aspect(0.5)

    # Video subplot for trial_a
    video_ax_a = fig.add_subplot(gs[1, 0])
    video_ax_a.axis("off")
    frame_image_a = video_ax_a.imshow(frames_a[0])  # Initialize with the first frame
    # Add a border to video_ax_a
    video_ax_a.add_patch(Rectangle((0, 0), frames_a[0].shape[1], frames_a[0].shape[0], 
                                linewidth=2, edgecolor='black', facecolor='none'))

    # Video subplot for trial_b
    video_ax_b = fig.add_subplot(gs[1, 1])
    video_ax_b.axis("off")
    frame_image_b = video_ax_b.imshow(frames_b[0])  # Initialize with the first frame

    # Add a border to video_ax_b
    video_ax_b.add_patch(Rectangle((0, 0), frames_b[0].shape[1], frames_b[0].shape[0], 
                                linewidth=2, edgecolor='black', facecolor='none'))
    # Function to update the plot and video frames
    def update(frame_idx):
        # Update the plots
        current_data_a = trial_a_data[trial_a_data["frame"] <= frame_idx]
        current_data_b = trial_b_data[trial_b_data["frame"] <= frame_idx]

        red_line_a.set_data(current_data_a["frame"], current_data_a["red"])
        green_line_a.set_data(current_data_a["frame"], current_data_a["green"])
        uncertain_line_a.set_data(current_data_a["frame"], current_data_a["uncertain"])

        red_line_b.set_data(current_data_b["frame"], current_data_b["red"])
        green_line_b.set_data(current_data_b["frame"], current_data_b["green"])
        uncertain_line_b.set_data(current_data_b["frame"], current_data_b["uncertain"])

        # Update the video frames
        if frame_idx < len(frames_a):
            frame_image_a.set_array(frames_a[frame_idx])
        if frame_idx < len(frames_b):
            frame_image_b.set_array(frames_b[frame_idx])

        return (red_line_a, green_line_a, uncertain_line_a,
                red_line_b, green_line_b, uncertain_line_b,
                frame_image_a, frame_image_b)

    # Create the animation
    ani = FuncAnimation(
        fig,
        update,
        frames=min(len(frames_a), len(frames_b)),
        interval=1000 / min(fps_a, fps_b),
        blit=False
    )

    # Convert animation to HTML5 video
    html_video = ani.to_html5_video().replace('loop>', '>')
    plt.close(fig)
    return HTML(html_video)

pairs = [name[:-1] for name in aggregated_df["global_trial_name"].unique() if name.endswith("a")]  # Base names of 'a' trials

for pair in pairs:
    trial_a = f"{pair}a"
    trial_b = f"{pair}b"

    print(f"Displaying Pair: {trial_a} and {trial_b}")

    trial_a_data = aggregated_df[aggregated_df["global_trial_name"] == trial_a]
    trial_b_data = aggregated_df[aggregated_df["global_trial_name"] == trial_b]
    animation_html = plot_pair_with_video(trial_a, trial_a_data, trial_b, trial_b_data)
    display(animation_html)


Displaying Pair: E1-1a and E1-1b


Displaying Pair: E1-2a and E1-2b


Displaying Pair: E1-3a and E1-3b


Displaying Pair: E1-4a and E1-4b


Displaying Pair: E1-5a and E1-5b


...

Displaying Pair: E2B-4a and E2B-4b


Displaying Pair: E2C-1a and E2C-1b


Displaying Pair: E2C-2a and E2C-2b


Displaying Pair: E2C-3a and E2C-3b


Displaying Pair: E2C-4a and E2C-4b
